# step 3 import data

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
path = r'/Users/tayen/Documents/CF/Instacart Basket Analysis'

In [4]:
ords_prods = pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','df_order_prods_labels.pickle'))

In [5]:
instacart = pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','instacart.pickle'))

In [18]:
ords_prods.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge1,...,_merge,price_range_loc,busiest_day,busiest_period_of_day,max_order,user_loyalty,avg_spending,spending_range,med_days_since,user_frequency
0,2539329,1,1,2,8,NaN,196,1,0,both,...,both,mid range product,regularly days,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customers
1,2398795,1,2,3,7,15.0,196,1,1,both,...,both,mid range product,lowest days,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customers
2,473747,1,3,3,12,21.0,196,1,1,both,...,both,mid range product,lowest days,Most orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customers
3,2254736,1,4,4,7,29.0,196,1,1,both,...,both,mid range product,lowest days,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customers
4,431534,1,5,4,15,28.0,196,1,1,both,...,both,mid range product,lowest days,Most orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customers


In [6]:
cstm = pd.read_csv(os.path.join(path,'02 Data','Original Data','customers.csv'), index_col = False)

In [7]:
cstm.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# step 4 wrangling data

In [8]:
# rename columns
cstm.rename({'First Name':'first_name','Surnam':'surname','Gender':'gender','STATE':'state','Age':'age'},axis = 1,inplace = True)

In [13]:
cstm.head()

,user_id,first_name,surname,gender,state,age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [9]:
cstm['name'] = cstm['first_name'] +' ' + cstm['surname']

In [10]:
# drop redundant columns 
cstm.drop(['first_name','surname'],axis = 1, inplace = True) 

In [11]:
cstm.head()

,user_id,gender,state,age,date_joined,n_dependants,fam_status,income,name
0,26711,Female,Missouri,48,1/1/2017,3,married,165665,Deborah Esquivel
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285,Patricia Hart
2,65803,Male,Idaho,35,1/1/2017,2,married,99568,Kenneth Farley
3,125935,Female,Iowa,40,1/1/2017,0,single,42049,Michelle Hicks
4,130797,Female,Maryland,26,1/1/2017,1,married,40374,Ann Gilmore


# step 5 check data quality

In [10]:
# check for mix-type data
for col in cstm.columns.tolist():
    weird = (cstm[[col]].applymap(type) != cstm[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (cstm[weird]) > 0:
     print(col)

name


In [12]:
# address mix-type data
cstm['name'] = cstm['name'].astype('str')

In [12]:
cstm['name'].describe()

count     206209
unique       947
top          nan
freq       11259
Name: name, dtype: object

In [ ]:
# check for missing date

In [38]:
cstm['user_id'].isnull().sum()

0

In [39]:
cstm['gender'].isnull().sum()

0

In [40]:
cstm['state'].isnull().sum()

0

In [41]:
cstm['age'].isnull().sum()

0

In [42]:
cstm['date_joined'].isnull().sum()

0

In [43]:
cstm['n_dependants'].isnull().sum()

0

In [44]:
cstm['fam_status'].isnull().sum()

0

In [45]:
cstm['income'].isnull().sum()

0

In [46]:
cstm['name'].isnull().sum()

0

In [ ]:
# remove for duplicats

In [50]:
cstm.shape

(206209, 9)

In [56]:
cstm_dups = cstm[cstm.duplicated()]

In [58]:
cstm_dups

,user_id,gender,state,age,date_joined,n_dependants,fam_status,income,name


In [13]:
cstm_clean = cstm.drop_duplicates()

In [52]:
cstm_clean.shape

(206209, 9)

# step 6 merge customer data

In [13]:
instacart_final = instacart.merge(cstm, on = 'user_id')

In [14]:
instacart_final.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge1,...,med_days_since,user_frequency,gender,state,age,date_joined,n_dependants,fam_status,income,name
0,2539329,1,1,2,8,NaN,196,1,0,both,...,20.5,Non-frequent customers,Female,Alabama,31,2/17/2019,3,married,40423,Linda Nguyen
1,2398795,1,2,3,7,15.0,196,1,1,both,...,20.5,Non-frequent customers,Female,Alabama,31,2/17/2019,3,married,40423,Linda Nguyen
2,473747,1,3,3,12,21.0,196,1,1,both,...,20.5,Non-frequent customers,Female,Alabama,31,2/17/2019,3,married,40423,Linda Nguyen
3,2254736,1,4,4,7,29.0,196,1,1,both,...,20.5,Non-frequent customers,Female,Alabama,31,2/17/2019,3,married,40423,Linda Nguyen
4,431534,1,5,4,15,28.0,196,1,1,both,...,20.5,Non-frequent customers,Female,Alabama,31,2/17/2019,3,married,40423,Linda Nguyen


In [15]:
instacart.drop(['_merge1','_merge'], axis = 1, inplace = True)

In [16]:
instacart.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_range_loc,busiest_day,busiest_period_of_day,max_order,user_loyalty,avg_spending,spending_range,med_days_since,user_frequency
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,9.0,mid range product,regularly days,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customers
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,mid range product,lowest days,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customers
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,mid range product,lowest days,Most orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customers
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,mid range product,lowest days,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customers
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,mid range product,lowest days,Most orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customers


# step 8 export merged data

In [17]:
instacart_final.to_pickle(os.path.join(path,'02 Data','Prepared Data','instacart_final.pickle'))